<a href="https://colab.research.google.com/github/AUT-Student/NN-HW5/blob/main/NN_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, Concatenate
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Dataset

In [ ]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
To: /content/lp1.data
100% 27.3k/27.3k [00:00<00:00, 464kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
To: /content/lp2.data
100% 14.6k/14.6k [00:00<00:00, 254kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
To: /content/lp3.data
100% 14.5k/14.5k [00:00<00:00, 253kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
To: /content/lp4.data
100% 33.6k/33.6k [00:00<00:00, 560kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data
To: /content/lp5.data
100% 49.2k/49.2k [00:00<00:00, 392kB/s]


In [ ]:
normal_label_datasets = ["normal","normal","ok","normal","normal"]

dataset_x = []
dataset_y = []
for i in range(5):
  normal_label = normal_label_datasets[i]
  dataset_i = pd.read_csv(f"/content/lp{i+1}.data", header=None)
  
  new_label = None
  new_data = None

  for j, row in dataset_i.iterrows():
    if j%16==0:
      new_data = []
      
      if row[0] == normal_label:
         new_label = 0
      else:
         new_label = 1

    else:
      new_data.append([float(item) for item in row[0].split()])

      if j%16==15:
        dataset_x.append(new_data)
        dataset_y.append(new_label)

In [ ]:
dataset_x, dataset_y = shuffle(dataset_x, dataset_y, random_state=0)

In [81]:
dataset_len = len(dataset_x)

In [83]:
train_x = dataset_x[:int(0.7*dataset_len)]
train_y = dataset_y[:int(0.7*dataset_len)]

valid_x = dataset_x[int(0.7*dataset_len):int(0.8*dataset_len)]
valid_y = dataset_y[int(0.7*dataset_len):int(0.8*dataset_len)]

test_x = dataset_x[int(0.8*dataset_len):]
test_y = dataset_y[int(0.8*dataset_len):]

# Elman Network

In [ ]:
class ElmanOld(keras.Model):
  def __init__(self):
    super().__init__(self)

    # inputs = Input(6)    
    # hidden = Dense(3)
    # output = Dense(2, activation="softmax")
    
    self.model = Sequential([
                             Input(8, name="Input"),
                             Dense(3, name="Hidden"),
                             Dense(2, activation="softmax", name="Output")
    ])
    
    self.x = K.constant([[0., 0.]])

    # x = output

    # self.model = keras.Model(inputs, output(inputs))

  def sub_call(self, input):
    print("sub call")
    print(f"input = {input}")
    print(f"self.x = {self.x}")
    # input = K.concatenate([tf.expand_dims(input, axis=0), self.x])
    input = tf.expand_dims(input, axis=0)
    concated = K.concatenate([input, self.x])
    output = self.model.call(concated)
    
    self.x = output

    return output

  def call(self, inputs):

    print(f"inputs = {inputs}")

    inputs = inputs[0]

    output = K.constant([[0., 0.]])
    for input in inputs:
      print(f"input = {input}")
      output = self.sub_call(input)
      print(f"output = {output}")

    return output

    # new_inputs_array = []
    # for input in inputs:
    #   new_inputs_array.append(K.concatenate([input, self.x]))
    
    # inputs = K.stack(new_inputs_array)

In [ ]:
class Elman(keras.Model):
  def __init__(self):
    super().__init__(self)

    self.model = Sequential([
                             Input(8, name="Input"),
                             Dense(3, name="Hidden"),
                             Dense(2, activation="softmax", name="Output")
    ])

  def sub_call(self, input, prev_input):
    if len(prev_input)==1:
      concated = K.concatenate([input, K.constant([0., 0.])])
      concated = tf.expand_dims(concated, axis=0)
    else:
      prev_output = self.sub_call(prev_input[-1], prev_input[:-1])

      input = tf.expand_dims(input, axis=0)
      concated = K.concatenate([input, prev_output])
    
    output = self.model.call(concated)  
    return output

  def call(self, inputs):
    inputs = inputs[0]

    input = inputs[-1]
    prev_input = inputs[:-1]

    return self.sub_call(input, prev_input)

In [ ]:
elman = Elman()

In [ ]:
elman.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [95]:
es_callback = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [96]:
elman.fit(x=train_x, y=train_y, epochs=20, batch_size=1, validation_data=(valid_x, valid_y), callbacks=[es_callback])

Epoch 1/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4498 - accuracy: 0.7346 - val_loss: 0.6211 - val_accuracy: 0.5870
Epoch 2/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.7407 - val_loss: 0.8124 - val_accuracy: 0.5217
Epoch 3/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4949 - accuracy: 0.7222 - val_loss: 0.6397 - val_accuracy: 0.6304
Epoch 4/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4575 - accuracy: 0.7284 - val_loss: 0.6136 - val_accuracy: 0.6087
Epoch 5/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4468 - accuracy: 0.7377 - val_loss: 0.6600 - val_accuracy: 0.6739
Epoch 6/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4788 - accuracy: 0.7160 - val_loss: 0.6470 - val_accuracy: 0.5870
Epoch 7/20
324/324 [==============================] - 1s 2ms/step - loss: 0.4502 - accuracy: 0.7377 - val_loss: 0.6140 - val_accuracy: 0.6304
Epoch 

In [99]:
elman.evaluate(x=train_x, y=train_y, batch_size=1)
elman.evaluate(x=valid_x, y=valid_y, batch_size=1)
elman.evaluate(x=test_x, y=test_y, batch_size=1)

93/93 [==============================] - 0s 2ms/step - loss: 0.4821 - accuracy: 0.6774


[0.48209214210510254, 0.6774193644523621]

In [101]:
len(test_y)

93

In [102]:
sum(test_y)

72